##### Imports

In [1]:
import sqlite3
from faker import Faker
import random

database_filename = "../gestionConferences.db"

##### Fill with random values

In [2]:
fake = Faker()

# --- Nombre d'entrées ---
NB_PERSONNES = 100
NB_UNIVERSITES = 5
NB_CONFERENCES = 100
NB_WORKSHOPS = 50
NB_MIN_SOUMISSIONS_PAR_CONFERENCE = 1
NB_MAX_SOUMISSIONS_PAR_CONFERENCE = 5
NB_MIN_SESSIONS_PAR_CONFERENCE = 1
NB_MAX_SESSIONS_PAR_CONFERENCE = 5
NB_RESPONSABLES = 30
NB_MIN_REPOS_PAR_CONF = 1
NB_MAX_RESPO_PAR_CONF = 4
NB_MIN_RESP_PAR_SESSION = 1
NB_MAX_RESP_PAR_SESSION = 4
NB_UTILISATEURS = 80

# --- Connexion à la base ---
conn = sqlite3.connect(database_filename)
cursor = conn.cursor()

In [3]:
# ------------------------
# TABLE PERSONNE
# ------------------------
personnes_data = []
for _ in range(NB_PERSONNES):
    nom = fake.last_name()
    prenom = fake.first_name()
    # Create email based on first and last name
    domain = random.choice(["yahoo.fr", "university.edu", "mail.com"])
    mail = f"{prenom.lower()}.{nom.lower()}@{domain}"
    personnes_data.append((nom, prenom, mail))

cursor.executemany(
    "INSERT INTO personne (name, first_name, mail) VALUES (?, ?, ?);",
    personnes_data
)

# Récupérer les ids créés
cursor.execute("SELECT id_personne FROM personne")
personnes_ids = [row[0] for row in cursor.fetchall()]

conn.commit()

In [4]:
# ------------------------
# TABLE UNIVERSITE
# ------------------------
university_names = ["ECL", "Lyon-1", "Lyon-2", "Lyon-3", "Paris-Saclay"]
universites_data = []
for i in range(NB_UNIVERSITES):
    name = university_names[i]
    adresse = fake.address()
    domain_name = ''.join(e for e in name.lower() if e.isalnum())
    mail = f"contact@{domain_name}.edu"
    universites_data.append((name, adresse, mail))

cursor.executemany(
    "INSERT INTO universite (name, adresse, mail) VALUES (?, ?, ?);",
    universites_data
)

cursor.execute("SELECT id_universite FROM universite")
universites_ids = [row[0] for row in cursor.fetchall()]

conn.commit()

In [5]:
# ------------------------
# TABLE CONFERENCE
# ------------------------
from datetime import timedelta, date
from utils.text_generation import generate_scientific_title, generate_scientific_intro
faker_dict = {
    "France": Faker("fr_FR"),
    "Allemagne": Faker("de_DE"),
    "Espagne": Faker("es_ES")
}
countries = list(faker_dict.keys())
weights = [0.8, 0.1, 0.1]

editors_list = [
    "Springer", "Elsevier", "IEEE", "ACM", "Wiley",
    "Taylor & Francis", "Nature"
]

# --- Step 1: generate normal conferences (not workshops) ---
normal_confs_data = []
for _ in range(NB_CONFERENCES):
    title = generate_scientific_title()
    
    # dates
    year = random.choice([2025, 2026])
    month = random.randint(1,12)
    day = random.randint(1,28)
    starting_date = date(year, month, day)
    ending_date = starting_date + timedelta(days=random.randint(1,5))
    
    # location
    country = random.choices(countries, weights=weights, k=1)[0]
    fake = faker_dict[country]
    city = fake.city()

    series = random.choice(["SDH", "CAISE", "TedTalk", ""])
    introduction = generate_scientific_intro()
    type_conf = "Conference"
    associated_conf = None
    # Split title into words, lowercase and strip punctuation
    words_in_title = [w.strip(".,").lower() for w in title.split()]

    # Remove unwanted stopwords
    stopwords = {"in", "of"}
    filtered_words = [w for w in words_in_title if w not in stopwords]

    # Select 1 to 4 keywords from the filtered list
    n_keywords = min(len(filtered_words), random.randint(1, 4))
    key_words = ','.join(random.sample(filtered_words, n_keywords))

    editor = random.choice(editors_list)
    id_universite = random.choice(universites_ids)
    
    normal_confs_data.append((
        title, starting_date.isoformat(), ending_date.isoformat(),
        city, country, series, introduction, type_conf,
        associated_conf, key_words, editor, id_universite
    ))

# Insert normal conferences and get their IDs
cursor.executemany(
    """INSERT INTO conference 
       (title, starting_date, ending_date, city, country, series, introduction,
        type, associated_conf, key_words, editor, id_universite)
       VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
    normal_confs_data
)
conn.commit()

cursor.execute("SELECT id_conference FROM conference WHERE type='Conference'")
normal_conf_ids = [row[0] for row in cursor.fetchall()]

# --- Step 2: generate workshops ---
workshops_data = []
for _ in range(NB_WORKSHOPS):
    title =  "Workshop : " + generate_scientific_title()

    # dates
    year = random.choice([2025, 2026])
    month = random.randint(1,12)
    day = random.randint(1,28)
    starting_date = date(year, month, day)
    ending_date = starting_date + timedelta(days=random.randint(1,5))
    
    # location
    country = random.choices(countries, weights=weights, k=1)[0]
    fake = faker_dict[country]
    city = fake.city()
    
    series = random.choice(["SDH", "CAISE", "TedTalk", ""])
    introduction = generate_scientific_intro()
    type_conf = "Workshop"
    # Choose associated_conf among existing normal conferences
    associated_conf = random.choice(normal_conf_ids)
    # Split title into words, lowercase and strip punctuation
    words_in_title = [w.strip(".,").lower() for w in title.split()]

    # Remove unwanted stopwords
    stopwords = {"in", "of", ":"}
    filtered_words = [w for w in words_in_title if w not in stopwords]

    # Select 1 to 4 keywords from the filtered list
    n_keywords = min(len(filtered_words), random.randint(1, 4))
    key_words = ','.join(random.sample(filtered_words, n_keywords))

    editor = fake.name()
    id_universite = random.choice(universites_ids)
    
    workshops_data.append((
        title, starting_date.isoformat(), ending_date.isoformat(),
        city, country, series, introduction, type_conf,
        associated_conf, key_words, editor, id_universite
    ))

# Insert workshops
cursor.executemany(
    """INSERT INTO conference 
       (title, starting_date, ending_date, city, country, series, introduction,
        type, associated_conf, key_words, editor, id_universite)
       VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
    workshops_data
)
conn.commit()

In [6]:
# ------------------------
# TABLE SOUMISSION
# ------------------------
cursor.execute("SELECT id_conference, type, starting_date FROM conference")
conferences = cursor.fetchall()  # [(id_conference, type, starting_date), ...]

soumissions_data = []
possible_soumissions_types = [
    "Regular paper",
    "Panel",
    "Tutorial",
    "Industrial development",
    "System descriptions",
    "Poster"
]

for conf_id, conf_type, conf_starting_date in conferences:
    conf_start_date = date.fromisoformat(conf_starting_date)  # convert string to date

    for _ in range(NB_MIN_SOUMISSIONS_PAR_CONFERENCE, NB_MAX_SOUMISSIONS_PAR_CONFERENCE):
        category = random.choice(possible_soumissions_types)
        if conf_type == "Workshop":
            category = "Workshop"
        
        # page number
        if category in ["Regular paper", "Poster", "Industrial development", "System descriptions"]:
            pages_number = random.randint(1, 5)
        else:
            pages_number = None  # No page number for Tutorial, Panel, Workshop

        # Layout
        if conf_type == "Workshop":
            layout = "Workshop format"
        elif category in ["Regular paper", "Panel", "Tutorial", "Poster"]:
            layout = f"Font size {random.randint(8,10)}, Arial"
        else:
            layout = random.choice(["PPT","PDF","LaTeX","Markdown","Google Slides"])

        # Dates: must be before conference start
        # Generate a random number of days before the conference (1–60)
        days_before = random.randint(1, 60)
        sending_date = conf_start_date - timedelta(days=random.randint(1, days_before))
        notif_date = sending_date - timedelta(days=random.randint(1, 10))
        deadline = notif_date - timedelta(days=random.randint(1, 10))

        soumissions_data.append((category, pages_number, layout, deadline.isoformat(), notif_date.isoformat(), sending_date.isoformat(), conf_id))

cursor.executemany(
    """INSERT INTO soumission
       (category, pages_number, layout, deadline, notif_date, sending_date, id_conference)
       VALUES (?, ?, ?, ?, ?, ?, ?)""",
    soumissions_data
)

conn.commit()


In [7]:
# ------------------------
# TABLE SESSION
# ------------------------
# First, fetch conference IDs and their key_words
cursor.execute("SELECT id_conference, key_words, title FROM conference")
conferences = cursor.fetchall()  # [(id_conference, key_words, title), ...]

sessions_data = []

for conf_id, key_words_str, conf_title in conferences:
    # Split the conference keywords into a list
    conf_keywords = [kw.strip() for kw in key_words_str.split(',') if kw.strip()]
    
    for i in range(NB_MIN_SESSIONS_PAR_CONFERENCE, NB_MAX_SESSIONS_PAR_CONFERENCE):
        title = f"Session {i} : {conf_title}"
        # Choose 1 or 2 keywords from the conference keywords for the session themes
        n_themes = min(len(conf_keywords), random.randint(1, 2))
        themes = ','.join(random.sample(conf_keywords, n_themes)) if conf_keywords else ''
        sessions_data.append((title, themes, conf_id))

cursor.executemany(
    "INSERT INTO session (title, themes, id_conference) VALUES (?, ?, ?)",
    sessions_data
)

# Fetch session IDs if needed
cursor.execute("SELECT id_session FROM session")
sessions_ids = [row[0] for row in cursor.fetchall()]

conn.commit()


In [8]:
# ------------------------
# TABLE RESPONSABLE
# ------------------------
responsables_data = []
responsable_types = [
    "Program Commitee chair",
    "Steering Commitee ",
    "Poster chair",
    "General chair",
    "Industrial session chair"
]

# Fetch personne info (id, name, first_name)
cursor.execute("SELECT id_personne, name, first_name FROM personne")
personnes = cursor.fetchall()  # [(id_personne, name, first_name), ...]

# Shuffle people to avoid duplicates
random.shuffle(personnes)

# Only take as many as NB_RESPONSABLES or the number of available people
for id_personne, name, first_name in personnes[:NB_RESPONSABLES]:
    # Professional email based on name
    pro_adress = f"{first_name.lower()}.{name.lower()}@work.com"
    
    type_resp = random.choice(responsable_types)
    responsables_data.append((pro_adress, type_resp, id_personne))

cursor.executemany(
    "INSERT INTO responsable (pro_adress, type, id_personne) VALUES (?, ?, ?)",
    responsables_data
)

cursor.execute("SELECT id_responsable FROM responsable")
responsables_ids = [row[0] for row in cursor.fetchall()]

conn.commit()


In [9]:
# ------------------------
# TABLE DIRECTION
# ------------------------
direction_data = []

# Keep track of existing pairs to avoid duplicates
existing_pairs = set()

cursor.execute("SELECT id_conference FROM conference")
conferences_ids = [row[0] for row in cursor.fetchall()]

for conf_id in conferences_ids:
    resp_sample = random.sample(responsables_ids, k=random.randint(NB_MIN_REPOS_PAR_CONF, NB_MAX_RESPO_PAR_CONF))
    for resp_id in resp_sample:
        pair = (conf_id, resp_id)
        if pair not in existing_pairs:
            direction_data.append(pair)
            existing_pairs.add(pair)  # mark as used

cursor.executemany(
    "INSERT INTO direction (id_conference, id_responsable) VALUES (?, ?)",
    direction_data
)

conn.commit()

In [10]:
# ------------------------
# TABLE EVALUATION
# ------------------------
evaluation_data = []

# Track existing (session, responsable) pairs to avoid duplicates
existing_pairs = set()

for session_id in sessions_ids:
    # Ensure each session has at least one responsable
    # Pick responsables randomly
    n_responsables = random.randint(NB_MIN_RESP_PAR_SESSION, NB_MAX_RESP_PAR_SESSION)
    resp_sample = random.sample(responsables_ids, k=n_responsables)
    
    for resp_id in resp_sample:
        pair = (session_id, resp_id)
        if pair not in existing_pairs:
            evaluation_data.append(pair)
            existing_pairs.add(pair)

# Optional: you could add more random assignments later, but each session already has >=1 responsable

cursor.executemany(
    "INSERT INTO evaluation (id_session, id_responsable) VALUES (?, ?)",
    evaluation_data
)

conn.commit()


In [11]:
# ------------------------
# TABLE UTILISATEUR
# ------------------------

utilisateurs_data = []

# Fetch all personnes IDs
cursor.execute("SELECT id_personne FROM personne")
personnes = [row[0] for row in cursor.fetchall()]

# Shuffle to ensure uniqueness
random.shuffle(personnes)

# Fetch all conference keywords
cursor.execute("SELECT key_words FROM conference")
all_conf_keywords = [
  "Neural Networks",
  "Quantum Computing",
  "Machine Learning",
  "Cybersecurity",
  "Data Mining",
  "Bioinformatics",
  "Robotics",
  "Climate Modeling",
  "Artificial Intelligence",
  "Software Engineering"
]

# Limit number of utilisateurs to number of available personnes
nb_to_create = min(NB_UTILISATEURS, len(personnes))

for id_personne in personnes[:nb_to_create]:
    # Choose 1-3 keywords as profile
    n_keywords = random.randint(1, min(3, len(all_conf_keywords)))
    profile_keywords = random.sample(all_conf_keywords, n_keywords)
    profile = ','.join(profile_keywords)
    
    utilisateurs_data.append((profile, id_personne))

cursor.executemany(
    "INSERT INTO utilisateur (profile, id_personne) VALUES (?, ?)",
    utilisateurs_data
)

conn.commit()


In [12]:
conn.close()